<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
!pip install --upgrade tensorflow

In [2]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

2023-05-25 04:51:54.580224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 04:51:54.784346: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 04:51:54.792326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-25 04:51:54.792374: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17464789/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

2023-05-25 04:52:00.815150: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-25 04:52:00.815218: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-25 04:52:00.815258: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-chongnating): /proc/driver/nvidia/version does not exist
2023-05-25 04:52:00.815673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 31s 38ms/step - loss: 0.6809 - accuracy: 0.5774 - val_loss: 0.6459 - val_accuracy: 0.6557
Epoch 2/10
782/782 [==============================] - 29s 37ms/step - loss: 0.6003 - accuracy: 0.6797 - val_loss: 0.5717 - val_accuracy: 0.6998
Epoch 3/10
782/782 [==============================] - 29s 38ms/step - loss: 0.5314 - accuracy: 0.7385 - val_loss: 0.5339 - val_accuracy: 0.7274
Epoch 4/10
782/782 [==============================] - 29s 37ms/step - loss: 0.4852 - accuracy: 0.7704 - val_loss: 0.5094 - val_accuracy: 0.7444
Epoch 5/10
782/782 [==============================] - 29s 37ms/step - loss: 0.4528 - accuracy: 0.7899 - val_loss: 0.4883 - val_accuracy: 0.7598
Epoch 6/10
782/782 [==============================] - 30s 39ms/step - loss: 0.4281 - accuracy: 0.8034 - val_loss: 0.4754 - val_accuracy: 0.7681
Epoch 7/10
782/782 [==============================] - 30s 38ms/step - loss: 0.4091 - accuracy: 0.8148 - val_loss: 0.4655 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 10s 13ms/step - loss: 0.4533 - accuracy: 0.7838
Test score: 0.45325151085853577
Test accuracy: 0.783840000629425


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 86s 109ms/step - loss: 0.6449 - accuracy: 0.6359 - val_loss: 0.5799 - val_accuracy: 0.7208
Epoch 2/10
782/782 [==============================] - 82s 105ms/step - loss: 0.5034 - accuracy: 0.7637 - val_loss: 0.4972 - val_accuracy: 0.7447
Epoch 3/10
782/782 [==============================] - 82s 105ms/step - loss: 0.4241 - accuracy: 0.8079 - val_loss: 0.4547 - val_accuracy: 0.7790
Epoch 4/10
782/782 [==============================] - 84s 108ms/step - loss: 0.3779 - accuracy: 0.8366 - val_loss: 0.4219 - val_accuracy: 0.8095
Epoch 5/10
782/782 [==============================] - 89s 114ms/step - loss: 0.3475 - accuracy: 0.8504 - val_loss: 0.4271 - val_accuracy: 0.7994
Epoch 6/10
782/782 [==============================] - 82s 105ms/step - loss: 0.3258 - accuracy: 0.8618 - val_loss: 0.3947 - val_accuracy: 0.8202
Epoch 7/10
782/782 [==============================] - 80s 103ms/step - loss: 0.3068 - accuracy: 0.8719 - val_loss: 0.3961 - val_ac

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 64s 80ms/step - loss: 0.6925 - accuracy: 0.5016 - val_loss: 0.6904 - val_accuracy: 0.5044
Epoch 2/10
782/782 [==============================] - 59s 76ms/step - loss: 0.6801 - accuracy: 0.5484 - val_loss: 0.6659 - val_accuracy: 0.6224
Epoch 3/10
782/782 [==============================] - 57s 72ms/step - loss: 0.6376 - accuracy: 0.6624 - val_loss: 0.6200 - val_accuracy: 0.7042
Epoch 4/10
782/782 [==============================] - 59s 76ms/step - loss: 0.5997 - accuracy: 0.7346 - val_loss: 0.5992 - val_accuracy: 0.7418
Epoch 5/10
782/782 [==============================] - 64s 82ms/step - loss: 0.5767 - accuracy: 0.7596 - val_loss: 0.5846 - val_accuracy: 0.7539
Epoch 6/10
782/782 [==============================] - 63s 81ms/step - loss: 0.5587 - accuracy: 0.7757 - val_loss: 0.5741 - val_accuracy: 0.7605
Epoch 7/10
782/782 [==============================] - 63s 81ms/step - loss: 0.5427 - accuracy: 0.7830 - val_loss: 0.5522 - val_accuracy:

In [ ]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 62s 80ms/step - loss: 0.4110 - accuracy: 0.8136 - val_loss: 0.4568 - val_accuracy: 0.7820
Epoch 2/10
782/782 [==============================] - 62s 79ms/step - loss: 0.3978 - accuracy: 0.8208 - val_loss: 0.4433 - val_accuracy: 0.7895
Epoch 3/10
782/782 [==============================] - 62s 79ms/step - loss: 0.3857 - accuracy: 0.8289 - val_loss: 0.4335 - val_accuracy: 0.7954
Epoch 4/10
782/782 [==============================] - 62s 79ms/step - loss: 0.3747 - accuracy: 0.8375 - val_loss: 0.4374 - val_accuracy: 0.7951
Epoch 5/10
378/782 [=============>................] - ETA: 27s - loss: 0.3649 - accuracy: 0.8399

---
### Machine Learning Foundation (C) 2020 IBM Corporation
